In [55]:
%pushd book-materials
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

# 챕터 기본 설정값은 저자가 생성해둔 깃허브 코드를 참조했다. 
# 링크: https://github.com/Ahnjunhyeok/pydata-book/blob/3rd-edition/ch08.ipynb


[WinError 2] 지정된 파일을 찾을 수 없습니다: 'book-materials'
c:\Users\ahnjh\Desktop\pythonWorkspace2\Jump_to_Python\Python_for_DA


In [56]:
# Chapter 8 데이터 준비하기: 조인, 병합, 변형

In [57]:
# 대부분의 경우 데이터는 여러 파일이나 데이터베이스 혹은 분석하기 쉽지 않은 형태로 기록되어 있다. 

In [58]:
# 이 장에서는 데이터를 합치고, 재배열할 수 있는 도구들을 살펴보자.

In [59]:
# 8.1 계층적 색인

In [60]:
# 계층적 색인은 pandas의 중요한 기능인데 축에 대해 다중(둘 이상) 색인 단계를 지정할 수 있도록 해준다. 

In [61]:
# 약간 추상적으로 말하면, 높은 차원의 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능이다. 

In [62]:
# 간단한 예제를 하나 살펴보자. 우선 리스트의 리스트(또는 배열)를 색인으로 하는 Series를 하나 생성하자. 

In [63]:
import numpy as np
import pandas as pd
import re
import sqlalchemy as sqla

In [64]:
data = pd.Series(np.random.randn(9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [65]:
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [66]:
# 지금 생성한 객체가 MultiIndex를 색인으로 하는 Series인데, 색인의 계층을 보여주고 있다. 

In [67]:
# 바로 위 단계의 색인을 이용해서 하위 계층을 직접 접근할 수 있다. 

In [93]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [69]:
# 계층적으로 색인된 객체는 데이터의 부분집합을 부분적 색인으로 접근하는 것이 가능하다. 

In [70]:
data["b"]

1   -0.555730
3    1.965781
dtype: float64

In [71]:
data["b":"c"]

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [72]:
data.loc[["b", "d"]]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [73]:
# 하위 계층의 객체를 선택하는 것도 가능하다. 

In [74]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [75]:
# 계층적인 색인은 데이터를 재형성하고 피벗테이블 생성 같은 그룹 기반의 작업을 할 때 중요하게 사용된다.

In [76]:
# 예를 들어 위에서 만든 DataFrame 객체에 unstack 메서드를 사용해서 데이터를 새롭게 배열할 수도 있다.

In [77]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [78]:
# unstack의 반대 작업은 stack 메서드로 수행한다.

In [79]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [80]:
# stack과 unstack 메서드는 이 장 후반부에서 더 자세히 알아보기로 하자. 

In [81]:
# DataFrame에서는 두 축 모두 계층적 색인을 가질 수 있다.

In [82]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])

In [83]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [84]:
# 계층적 색인의 각 단계는 이름(문자열이나 어떤 파이썬 객체라도 가능하다)을 가질 수 있고, 만약 이름을 가지고 있다면 콘솔 출력 시 함께 나타난다.

In [85]:
frame.index.names = ["key1", "key2"]

In [86]:
frame.columns.names = ["state", "color"]

In [87]:
frame # Caution! 색인 이름인 "state"와 "color"를 로우 라벨과 혼동하지 말자.

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [94]:
frame.index.nlevels

2

In [88]:
# 컬럼의 부분집합을 부분적인 색인으로 접근하는 것도 컬럼에 대한 부분적 색인과 비슷하게 사용하면 된다.

In [89]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [90]:
# MultiIndex는 따로 생성한 다음에 재사용 가능하다. 위에서 살펴본 DataFrame의 컬럼 계층 이름은 다음처럼 생성할 수 있다.

In [95]:
# MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]],
#                         names=["State", "color"])

In [96]:
# 8.1.1 계층의 순서를 바꾸고 정렬하기

In [97]:
# 계층적 색인에서 계층의 순서를 바꾸거나 지정된 계층에 따라 데이터를 정렬해야 하는 경우가 있을 수 있다.

In [98]:
# swaplevel은 넘겨받은 두 개의 계층 번호나 이름이 뒤바뀐 새로운 객체를 반환한다(하지만 데이터는 변경되지 않는다)

In [99]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [100]:
# 반면 sort_index 메서드는 단일 계층에 속한 데이터를 정렬한다. 

In [101]:
# swaplevel을 이용해서 계층을 바꿀 때 sort_index를 사용해서 결과가 사전적으로 정렬되도록 만드는 것도 드물지 않은 일이다. 

In [102]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [103]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [104]:
# Note_ 객체가 계층적 색인으로 상위 계층부터 사전식으로 정렬되어 있다면(sort_index(level=0)이나 sort_index()의 결과처럼) 데이터를 선택하는 성능이 훨씬 좋아진다.

In [105]:
# 8.1.2 계층별 요약 통계

In [106]:
# DataFrame과 Series의 많은 기술 통계와 요약 통계는 level 옵션을 가지고 있는데, 어떤 한 축에 대해 합을 구하고 싶은 단계를 지정할 수 있는 옵션이다.

In [107]:
# 앞에서 살펴본 DataFrame에서 로우나 컬럼을 아래처럼 계층별로 합할 수 있다.

In [109]:
frame.sum(level="key2")

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_35476\3430452342.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level="key2")


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [110]:
frame.sum(level="color", axis=1)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_35476\3066880672.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level="color", axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [111]:
# 이는 내부적으로 pandas의 groupby 기능을 이용해서 구현되었는데, 자세한 내용은 앞으로 더 살펴보겠다.

In [112]:
# 8.1.3 DataFrame의 컬럼 사용하기

In [113]:
# DataFrame에서 로우를 선택하기 위한 색인으로 하나 이상의 컬럼을 사용하는 것은 드물지 않은 일이다. 

In [114]:
# 아니면 로우의 색인을 DataFrame의 컬럼으로 옮기고 싶을 것이다. 다음은 예제 DataFrame이다.

In [115]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

In [116]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [117]:
# DataFrame의 set_index 함수는 하나 이상의 컬럼을 색인으로 하는 새로운 DataFrame을 생성한다. 

In [118]:
frame2 = frame.set_index(["c", "d"])

In [119]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [120]:
# 다음처럼 컬럼을 명시적으로 남겨두지 않으면 DataFrame에서 삭제된다.

In [121]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [122]:
# 반면 reset_index 함수는 set_index와 반대되는 개념인데 계층적 색인 단계가 컬럼으로 이동한다. 

In [123]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [124]:
# 8.2 데이터 합치기

In [125]:
# pandas 객체에 저장된 데이터는 여러 가지 방법으로 합칠 수 있다.

In [126]:
# 1. pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터베이스의 join 연산과 유사하다.
# 2. pandas.concat은 하나의 축을 따라 객체를 이어붙인다.
# 3. combile_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다. 

In [127]:
# 각각의 데이터를 합치는 방법에 대해 다양한 예제와 함께 살펴보게 될 것이다. 이 기법은 책 전체에서 계속 활용된다.

In [128]:
# 8.2.1 데이터베이스 스타일로 DataFrame 합치기

In [129]:
# merge(병합)나 join(조인) 연산은 관계형 데이터베이스의 핵심적인 연산인데, 하나 이상의 키를 사용해서 데이터 집합의 로우를 합친다.

In [131]:
# pandas의 merge 함수를 이용해서 이런 알고리즘을 데이터에 적용할 수 있다. 예제를 살펴보자

In [132]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": range(7)})

In [133]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": range(3)})

In [134]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [135]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [136]:
# 위 예제는 '다대일'의 경우다. df1의 데이터는 key 컬럼에 여러 개의 a, b 로우를 가지고 있고 df2의 데이터는 key 컬럼에 유일한 로우를 가지고 있다.

In [138]:
# 이 객체에 대해 merge 함수를 호출하면 다음과 같은 결과를 얻는다.

In [139]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [140]:
# 위에서 나는 어떤 컬럼을 병합할 것인지 명시하지 않았는데, merge 함수는 중복된 컬럼 이름을 키로 사용한다(위 예에서는 key 컬럼).

In [141]:
# 하지만 명시적으로 지정해주는 습관을 들이는 게 좋다.

In [142]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [143]:
# 만약 두 객체에 중복된 컬럼 이름이 하나도 없다면 따로 지정해주면 된다.

In [144]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": range(7)})

In [145]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": range(3)})

In [146]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [147]:
# 결과를 잘 살펴보면 "c"와 "d"에 해당하는 값이 빠진 것을 알 수 있다. 

In [148]:
# merge 함수는 기본적으로 내부 조인(inner join)을 수행하여 교집합인 결과를 반환한다. 

In [149]:
# how 인자로 "left", "right", "outer"를 넘겨서 각각 왼쪽 조인, 오른쪽 조인, 외부 조인을 수행할 수도 있다. 

In [150]:
# 외부 조인은 합집합인 결과를 반환하고 왼쪽 조인과 오른쪽 조인은 각각 왼쪽 또는 오른쪽의 모든 로우를 포함하는 결과를 반환한다. 

In [151]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [153]:
# [표 8-1]에 how 옵션에 따라 조인 연산이 어떻게 동작하는지 요약해두었다. 페이지 317

In [154]:
# "다대다" 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다. 

In [155]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": range(6)})

In [156]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": range(5)})

In [157]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [158]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [159]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [160]:
# 다대다 조인은 두 로우의 데카르트곱을 반환한다. 

In [161]:
# 왼쪽 DataFrame에는 3개의 "b" 로우가 있고 오른쪽에는 2개의 "b" 로우가 있으며, 결과는 6개의 "b" 로우가 된다. 

In [162]:
# 조인 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용된다. 

In [163]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [164]:
# 여러 개의 키를 병합하려면 컬럼 이름이 담긴 리스트를 넘기면 된다. 

In [165]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": [1, 2, 3]})

In [166]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": [4, 5, 6, 7]})

In [167]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [168]:
# merge 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알려면 실제 구현과는 조금 다르지만 여러 개의 키가 들어 있는 튜플의 배열이 단일 조인키로 사용된다고 생각하면 된다. 

In [169]:
# Caution! 컬럼과 컬럼을 조인할 때 DataFrame 객체의 색인은 무시된다. 

In [170]:
# 병합 연산에서 고려해야 할 마지막 사항은 겹치는 컬럼 이름에 대한 처리다. 

In [171]:
# 다음에 살펴보겠지만 축 이름을 변경해서 수동으로 컬럼 이름이 겹치게 할 수도 있고, merge 함수에 있는 suffixes 인자로 두 DataFrame 객체에서 겹치는 컬럼 이름 뒤에 붙일 문자열을 지정해줄 수도 있다.

In [173]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [174]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [175]:
# [표 8-2]에 merge 함수의 인자를 나타냈다. 다음 절에서는 DataFrame의 로우 색인에 대한 조인을 알아본다. 페이지 320-321

In [176]:
# 8.2.2 색인 변형하기

In [177]:
# 병합하려는 키가 DataFrame의 색인일 경우가 있다. 

In [178]:
# 이런 경우에는 left_index=True 혹은 right_index=True 옵션(또는 둘 다)을 지정해서 해당 색인을 병합키로 사용할 수 있다. 

In [180]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                     "value": range(6)})

In [181]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [182]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [183]:
right1

,group_val
a,3.5
b,7.0


In [184]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [185]:
# 병합은 기본적으로 교집합을 구하지만 외부 조인을 실행해서 합집합을 구할 수도 있다. 

In [186]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [187]:
# 계층 색인된 데이터는 암묵적으로 여러 키를 병합하는 것이라 약간 복잡하다. 

In [188]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": np.arange(5.)})

In [189]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[["Nevada", "Nevada", "Ohio", "Ohio",
                              "Ohio", "Ohio"],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                             columns=["event1" ,"event2"])

In [195]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [196]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [197]:
# 이 경우에는 리스트로 여러 개의 컬럼을 지정해서 병합해야 한다(중복되는 색인값을 다룰 때는 how="outer" 옵션을 사용해야 한다).

In [198]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [199]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [200]:
# 양쪽에 공통적으로 존재하는 여러 개의 색인을 병합하는 것도 가능하다.

In [201]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"], 
                     columns=["Ohio", "Nevada"])

In [202]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"])

In [203]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [204]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [205]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [206]:
# 색인으로 병합할 때 DataFrame의 join 메서드를 사용하면 편리하다. 

In [207]:
# join 메서드는 컬럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다. 
# 위에서 본 예제는 다음처럼 작성할 수 있다. 

In [208]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [209]:
# 과거에 작성된 pandas의 일부 코드 제약으로 인해 DataFrame의 join 메서드는 왼쪽 조인을 수행한다. 

In [210]:
# join 메서드를 호출한 DataFrame의 컬럼 중 하나에 대해 조인을 수행하는 것도 가능하다. 

In [211]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [212]:
# 마지막으로 색인 대 색인으로 두 DataFrame을 병합하려면 그냥 간단하게 병합하려는 DataFrame의 리스트를 join 메서드로 넘기면 된다. 

In [213]:
# 하지만 보통 이런 병합은 아래에서 살펴볼 concat 메서드를 사용한다. 

In [214]:
another = pd.DataFrame([[7., 8.], [9., 10], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"], 
                       columns=["New York", "Oregon"])

In [215]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [216]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [221]:
left2.join([right2 , another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0
